In [1]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
import re
load_dotenv()
from pathlib import Path
from nltk.corpus import reuters, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from wordcloud import WordCloud

In [32]:
tweepyKey = os.getenv("tweepyKey")
tweepyClient = tweepy.Client(tweepyKey)
tweepyClient = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAGvbawEAAAAAxvxnJu9E%2F%2FPCe5Oz0irJC5BDvZM%3DEsqky8zPsYhadIeHKzCVsaB0UXGz28FFb2UBqtnNDOJkmfAkfN")

startDate = '2022-05-31'
endDate = '2022-06-01'

In [3]:
# import nltk
# nltk.download('vader_lexicon')
# from textblob import TextBlob as tb
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sid = SentimentIntensityAnalyzer()
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2022-06-06 19:08:14,048 loading file /Users/guruprasadmulay/.flair/models/sentiment-en-mix-distillbert_4.pt


In [5]:
file_path = Path("Resources/BTC_Large.csv")
BTCTweets = pd.read_csv(file_path)
BTCTweets.head()

,time,tweet
0,2022-05-31 00:03:52+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000..."
1,2022-05-31 00:03:53+00:00,RT @Next100XGEMS: I’m so #Bullish on #BTC 📈👀
2,2022-05-31 00:03:53+00:00,@cz_binance @BTC_Archive @wolfofcrypto89 you a...
3,2022-05-31 00:03:53+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000..."
4,2022-05-31 00:03:54+00:00,RT @RoyalBlackCard: THE LEADING UTILITY NFT 🏆\...


In [6]:
#cleaning
def cleaning(df):
    cleaned_tweets=[]
    for word in df['tweet']:
        #convert to string
        tw=str(word)
        #remove web links
        tw=re.sub(r"http\S+", "",tw)
        tw=re.sub(r"www\S+","",tw)
        #Remove mentions
        tw=re.sub(r"@\S+", "", tw)
        #remove non alphabetic characters
        tw=re.sub("[^A-Za-z0-9]"," ",tw)
        #remove rt
        tw=re.sub(r"RT","",tw)
        cleaned_tweets.append(tw.lower())
    df['cleaned_tweets']=cleaned_tweets
    return

#tokenizer 
def tokenizer(df):
    sw = set(stopwords.words('english'))
    token_tweets=[]
    for tweet in df['cleaned_tweets']:
        words = word_tokenize(tweet)
        lem = [lemmatizer.lemmatize(word) for word in words]
        output=[word for word in lem if word not in sw]
        token_tweets.append(output)
    df['token_tweets']=token_tweets
    return

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def sentiment(df):
    compound=[]
    pos=[]
    neu=[]
    neg=[]
    for tweet in df['cleaned_tweets']:
        sentiment=analyzer.polarity_scores(tweet)
        compound.append(sentiment['compound'])
        pos.append(sentiment['pos'])
        neu.append(sentiment['neu'])
        neg.append(sentiment['neg'])
    df['compound']=compound
    df['pos']=pos
    df['neu']=neu
    df['neg']=neg
    return
import matplotlib.pyplot as plt

def wc(df):
    biglist=[]
    for tweets in df['token_tweets']:
        biglist=biglist+tweets
    bigstring=' '.join(biglist)
    wc=WordCloud().generate(bigstring)
    return plt.imshow(wc)


In [7]:
cleaning(BTCTweets)
#  TODO include word embedding instead of tokenizer
tokenizer(BTCTweets)
sentiment(BTCTweets)



In [8]:
BTCTweets['positive'] = 0
BTCTweets.loc[BTCTweets['pos'] > 0, 'positive'] = 1
BTCTweets['negative'] = 0
BTCTweets.loc[BTCTweets['neg'] > 0, 'negative'] = 1
BTCTweets.head(20)

,time,tweet,cleaned_tweets,token_tweets,compound,pos,neu,neg,positive,negative
0,2022-05-31 00:03:52+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000...",bitcoin has not reached 50 000 so i will ...,"[bitcoin, ha, reached, 50, 000, 2, 500, giveaw...",0.8128,0.373,0.577,0.05,1,1
1,2022-05-31 00:03:53+00:00,RT @Next100XGEMS: I’m so #Bullish on #BTC 📈👀,i m so bullish on btc,"[bullish, btc]",0.0000,0.000,1.000,0.00,0,0
2,2022-05-31 00:03:53+00:00,@cz_binance @BTC_Archive @wolfofcrypto89 you a...,you are a kahbesinnnn,[kahbesinnnn],0.0000,0.000,1.000,0.00,0,0
3,2022-05-31 00:03:53+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000...",bitcoin has not reached 50 000 so i will ...,"[bitcoin, ha, reached, 50, 000, 2, 500, giveaw...",0.8128,0.373,0.577,0.05,1,1
4,2022-05-31 00:03:54+00:00,RT @RoyalBlackCard: THE LEADING UTILITY NFT 🏆\...,the leading utility nft kyc audited t...,"[leading, utility, nft, kyc, audited, early, b...",0.0000,0.000,1.000,0.00,0,0
5,2022-05-31 00:03:54+00:00,RT @RoyalBlackCard: THE LEADING UTILITY NFT 🏆\...,the leading utility nft kyc audited t...,"[leading, utility, nft, kyc, audited, early, b...",0.0000,0.000,1.000,0.00,0,0
6,2022-05-31 00:03:54+00:00,RT @RoyalBlackCard: THE LEADING UTILITY NFT 🏆\...,the leading utility nft kyc audited t...,"[leading, utility, nft, kyc, audited, early, b...",0.0000,0.000,1.000,0.00,0,0
7,2022-05-31 00:03:54+00:00,RT @AirdropDet: 🔍 New #Airdrop: Swords of Bloo...,new airdrop swords of blood reward ...,"[new, airdrop, sword, blood, reward, 222, swdt...",0.5719,0.171,0.829,0.00,1,0
8,2022-05-31 00:03:54+00:00,RT @LunaBurnx: #LUNABURNX 🌕🔥\n\n🚀JUNE 1ST: #Pr...,lunaburnx june 1st presale amp la...,"[lunaburnx, june, 1st, presale, amp, launch, p...",-0.3818,0.000,0.860,0.14,0,1
9,2022-05-31 00:03:54+00:00,RT @BTC_Archive: #Bitcoin will be the 'reserve...,bitcoin will be the reserve currency of t...,"[bitcoin, reserve, currency, world, kevin, leary]",0.0000,0.000,1.000,0.00,0,0


In [38]:
# BTCTweets
BTCTweets[BTCTweets['compound'] != 0]

,time,tweet,cleaned_tweets,token_tweets,compound,pos,neu,neg,positive,negative
0,2022-05-31 00:03:52+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000...",bitcoin has not reached 50 000 so i will ...,"[bitcoin, ha, reached, 50, 000, 2, 500, giveaw...",0.8128,0.373,0.577,0.050,1,1
3,2022-05-31 00:03:53+00:00,"RT @rovercrc: #Bitcoin has not reached $50,000...",bitcoin has not reached 50 000 so i will ...,"[bitcoin, ha, reached, 50, 000, 2, 500, giveaw...",0.8128,0.373,0.577,0.050,1,1
7,2022-05-31 00:03:54+00:00,RT @AirdropDet: 🔍 New #Airdrop: Swords of Bloo...,new airdrop swords of blood reward ...,"[new, airdrop, sword, blood, reward, 222, swdt...",0.5719,0.171,0.829,0.000,1,0
8,2022-05-31 00:03:54+00:00,RT @LunaBurnx: #LUNABURNX 🌕🔥\n\n🚀JUNE 1ST: #Pr...,lunaburnx june 1st presale amp la...,"[lunaburnx, june, 1st, presale, amp, launch, p...",-0.3818,0.000,0.860,0.140,0,1
12,2022-05-31 00:03:55+00:00,Someone said #Bitcoin will not go below 30k an...,someone said bitcoin will not go below 30k an...,"[someone, said, bitcoin, go, 30k, want, believe]",0.0772,0.098,0.902,0.000,1,0
...,...,...,...,...,...,...,...,...,...,...
398774,2022-06-05 23:59:58+00:00,RT @Revolotto: 🇲🇽The President of the world's ...,the president of the world s most powerful...,"[president, world, powerful, country, beloved,...",0.9134,0.476,0.524,0.000,1,0
398775,2022-06-05 23:59:59+00:00,RT @AirdropStario: 💧 DogeFuu Airdrop 💧\n\n🏆 Ta...,dogefuu airdrop task 3 worth of d...,"[dogefuu, airdrop, task, 3, worth, dogefuu, 30...",0.4215,0.213,0.787,0.000,1,0
398776,2022-06-05 23:59:59+00:00,Can #Bitcoin Become “One Of The Best Assets On...,can bitcoin become one of the best assets on...,"[bitcoin, become, one, best, asset, earth, exp...",0.7096,0.312,0.688,0.000,1,0
398777,2022-06-05 23:59:59+00:00,Getting paid crypto to create whatever content...,getting paid crypto to create whatever content...,"[getting, paid, crypto, create, whatever, cont...",0.3400,0.159,0.841,0.000,1,0


In [42]:
# remov na compounds
BTCTweets['time'] = pd.DatetimeIndex(BTCTweets['time'])
BTCTweetsTemp = BTCTweets[BTCTweets['compound'] != 0]
BTCTweetsTemp = BTCTweetsTemp[['time','compound']]
BTCTweetsTemp

,time,compound
0,2022-05-31 00:03:52+00:00,0.8128
3,2022-05-31 00:03:53+00:00,0.8128
7,2022-05-31 00:03:54+00:00,0.5719
8,2022-05-31 00:03:54+00:00,-0.3818
12,2022-05-31 00:03:55+00:00,0.0772
...,...,...
398774,2022-06-05 23:59:58+00:00,0.9134
398775,2022-06-05 23:59:59+00:00,0.4215
398776,2022-06-05 23:59:59+00:00,0.7096
398777,2022-06-05 23:59:59+00:00,0.3400


In [43]:

# BTCTweetsTemp2=BTCTweetsTemp
# convert date format to hour & minutes
# BTCTweetsTemp2['date'] = pd.to_datetime(BTCTweetsTemp2.index.strftime("%Y-%m-%d %-H:%M"))

# get avg compound score for minutes interval
# BTCTweetsTemp1 = BTCTweetsTemp2 \
# .groupby(by=[ BTCTweetsTemp2.date ]).mean()
BTCTweetsTemp1 = BTCTweetsTemp.groupby(pd.Grouper(key="time", freq='T'),
     as_index=True
).mean()
BTCTweetsTemp1

,compound
time,
2022-05-31 00:03:00+00:00,0.370121
2022-05-31 00:04:00+00:00,0.263006
2022-05-31 00:05:00+00:00,NaN
2022-05-31 00:06:00+00:00,0.281656
2022-05-31 00:07:00+00:00,NaN
...,...
2022-06-05 23:55:00+00:00,0.458326
2022-06-05 23:56:00+00:00,NaN
2022-06-05 23:57:00+00:00,0.511036


In [44]:
BTCTweetsCount = BTCTweets[['time', 'positive', 'negative']].groupby(pd.Grouper(key="time", freq='T'),
     as_index=True).sum()
BTCTweetsCount

,positive,negative
time,,
2022-05-31 00:03:00+00:00,11,6
2022-05-31 00:04:00+00:00,27,18
2022-05-31 00:05:00+00:00,0,0
2022-05-31 00:06:00+00:00,41,18
2022-05-31 00:07:00+00:00,0,0
...,...,...
2022-06-05 23:55:00+00:00,61,22
2022-06-05 23:56:00+00:00,0,0
2022-06-05 23:57:00+00:00,74,26


In [74]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
tradesData = pdr.get_data_yahoo("BTC-USD", start='2022-05-31', end='2022-06-06', interval="1m")

# tradesData['actual_returns'] = tradesData['Close'].pct_change()
# tradesData.tail()
# tradesData[['actual_returns']]

[*********************100%***********************]  1 of 1 completed


In [82]:
merged_df = pd.concat([BTCTweetsCount, BTCTweetsTemp1], join="inner", axis=1)
merged_df = pd.concat([merged_df, tradesData], join="inner", axis=1)
merged_df['actual_returns'] = merged_df['Close'].pct_change()
merged_df = merged_df.dropna()
merged_df['target_return'] = merged_df['actual_returns'].shift(-1)
# final_df = merged_df.dropna()
# final_df
merged_df = merged_df[['positive', 'negative', 'compound', 'actual_returns', 'target_return']]
# final_df = merged_df.dropna()
# final_df
final_df = merged_df

# final_df.to_csv('Resources/final_df_without_y')
file_path = Path("Resources/final_df_without_y.csv")
final_df.to_csv(file_path)

In [79]:

final_df['y'] = 0
# if targe return is greater than 0 buy
final_df.loc[final_df['target_return'] > 0, 'y'] = 1
final_df.loc[final_df['target_return'] < 0, 'y'] = -1
final_df


,positive,negative,compound,actual_returns,target_return,y
2022-05-31 00:04:00+00:00,27,18,0.263006,0.000528,0.000117,1
2022-05-31 00:06:00+00:00,41,18,0.281656,0.000418,0.000710,1
2022-05-31 00:08:00+00:00,45,36,0.149611,0.000285,-0.000368,-1
2022-05-31 00:10:00+00:00,53,26,0.314945,0.000119,0.000103,1
2022-05-31 00:12:00+00:00,65,30,0.418451,0.000544,-0.001238,-1
...,...,...,...,...,...,...
2022-06-05 13:52:00+00:00,41,14,0.465333,-0.000191,-0.000028,-1
2022-06-05 13:53:00+00:00,39,7,0.624587,-0.000028,-0.000476,-1
2022-06-05 13:55:00+00:00,81,36,0.541112,-0.000897,-0.000347,-1
2022-06-05 13:57:00+00:00,75,35,0.528340,0.000123,-0.000563,-1
